In [1]:
import sys
import os
sys.path.append(os.path.split(os.getcwd())[0])

In [2]:
from src import dataset
from src.models import DNN, LSTM, GRU

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
symbol = "^SPX"

In [5]:
ds = dataset.create_dataset(symbol)

In [6]:
x_train, y_train = ds["train"]["X"], ds["train"]["Y"]
print(x_train.shape)
print(y_train.shape)

(2748, 20)
(2748,)


In [7]:
batch_size = 4
lags = 5

In [8]:
x_train = torch.tensor(x_train[:, :lags], dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

In [9]:
model = LSTM(input_shape=x_train.shape[1]).to(device)

In [10]:
criterion = nn.BCELoss()

# Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=False)

In [12]:
for input, label in train_loader:
    break

In [13]:
input.shape

torch.Size([4, 5])

In [14]:
X = input.to(device)
y_true = label.to(device)

optimizer.zero_grad()

In [15]:
print(model)

LSTM(
  (net): Sequential(
    (0): Linear(in_features=32, out_features=16, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): Linear(in_features=16, out_features=1, bias=True)
  )
  (reccurent): ModuleList(
    (0): LSTM(5, 32, num_layers=2, dropout=0.2)
    (1): LSTM(32, 32, num_layers=2, dropout=0.2)
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (sigmoid): Sigmoid()
)


In [16]:
model(X)
model.hidden

(tensor([[-0.1922,  0.0478, -0.0189, -0.0098,  0.0178,  0.0737, -0.0291,  0.0385,
           0.0727, -0.0542,  0.0361, -0.0606,  0.0439,  0.0045,  0.1584, -0.0224,
          -0.1351, -0.0200,  0.0760, -0.2726,  0.1285,  0.0996,  0.0012, -0.0150,
          -0.1155,  0.0003, -0.0674,  0.0784,  0.1259, -0.1522, -0.1641,  0.1647],
         [ 0.0861,  0.0035,  0.0583,  0.1055, -0.0346,  0.0332,  0.0051,  0.0313,
          -0.1028, -0.0722,  0.0339,  0.0271, -0.0599,  0.0008,  0.0038, -0.0352,
          -0.0367,  0.1260,  0.0562,  0.0473,  0.0610,  0.0524, -0.0411, -0.0632,
           0.0482, -0.1045,  0.1444, -0.1712,  0.0453, -0.1188,  0.0973, -0.0153]],
        device='cuda:0', grad_fn=<SqueezeBackward1>),
 tensor([[-3.6044e-01,  1.1729e-01, -3.5200e-02, -2.1730e-02,  3.5623e-02,
           1.7167e-01, -6.4276e-02,  8.3575e-02,  1.8222e-01, -1.4028e-01,
           7.8848e-02, -1.1623e-01,  8.1784e-02,  8.5280e-03,  2.9741e-01,
          -3.9674e-02, -2.7022e-01, -3.6582e-02,  1.7450e-01, 

In [19]:
for state in model.hidden:
    print(state.data)

tensor([[-0.1922,  0.0478, -0.0189, -0.0098,  0.0178,  0.0737, -0.0291,  0.0385,
          0.0727, -0.0542,  0.0361, -0.0606,  0.0439,  0.0045,  0.1584, -0.0224,
         -0.1351, -0.0200,  0.0760, -0.2726,  0.1285,  0.0996,  0.0012, -0.0150,
         -0.1155,  0.0003, -0.0674,  0.0784,  0.1259, -0.1522, -0.1641,  0.1647],
        [ 0.0861,  0.0035,  0.0583,  0.1055, -0.0346,  0.0332,  0.0051,  0.0313,
         -0.1028, -0.0722,  0.0339,  0.0271, -0.0599,  0.0008,  0.0038, -0.0352,
         -0.0367,  0.1260,  0.0562,  0.0473,  0.0610,  0.0524, -0.0411, -0.0632,
          0.0482, -0.1045,  0.1444, -0.1712,  0.0453, -0.1188,  0.0973, -0.0153]],
       device='cuda:0')
tensor([[-3.6044e-01,  1.1729e-01, -3.5200e-02, -2.1730e-02,  3.5623e-02,
          1.7167e-01, -6.4276e-02,  8.3575e-02,  1.8222e-01, -1.4028e-01,
          7.8848e-02, -1.1623e-01,  8.1784e-02,  8.5280e-03,  2.9741e-01,
         -3.9674e-02, -2.7022e-01, -3.6582e-02,  1.7450e-01, -5.5379e-01,
          2.9303e-01,  1.6602

In [52]:
model.reset_hidden()

In [54]:
model.hidden